# 01 - Bulletproofing your data

[Nameparser library](https://pypi.org/project/nameparser/)

In [159]:
# import libraries
import pandas as pd
from nameparser import HumanName

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

### Practice
#### FED board of directors 
##### 1. Download
- [Diversity in the FED - Brookings Institute](https://www.brookings.edu/research/diversity-within-the-federal-reserve-system/): download file named " Biographical database by unique position" on lefthand side
- [Documentation](https://www.brookings.edu/wp-content/uploads/2021/04/Biographical-Database-Overview.pdf): go through each column to understand what's there
- [Brookings Analysis](https://www.brookings.edu/wp-content/uploads/2021/04/Appendices-Directors-by-race-gender-and-bank.pdf)

In [108]:
# get excel file from assets/ folder
df_bod = pd.read_excel('downloaded_data/Biographical-database-BoD-Unique-Positions.xlsx')

In [109]:
# check row and column lengths
df_bod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2607 entries, 0 to 2606
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  2607 non-null   object 
 1   District Number       2607 non-null   int64  
 2   District Name         2607 non-null   object 
 3   Class                 2607 non-null   object 
 4   Group                 1393 non-null   float64
 5   TD1: Degree           2607 non-null   object 
 6   TD1: Major/Field      2607 non-null   object 
 7   TD1: School           2607 non-null   object 
 8   TD1: Year             2607 non-null   object 
 9   TD2: Degree           2607 non-null   object 
 10  TD2: Major/Field      2607 non-null   object 
 11  TD2: School           2607 non-null   object 
 12  TD2: Year             2606 non-null   object 
 13  City                  2606 non-null   object 
 14  State                 2607 non-null   object 
 15  Job Title            

In [110]:
# trim columns and covert all to strings
df_bod = df_bod.applymap(lambda x: x.strip() if type(x) == str else str(x))

In [116]:
# standardize null values
df_bod = df_bod.applymap(lambda x: np.nan if str(x) in ['-', '?', 'None', 'nan'] else x)

In [117]:
# check columns: with missing values (Group)
df_bod['Group'].unique(), df_bod['Class'].unique()

(array(['3.0', nan, '2.0', '1.0'], dtype=object),
 array(['A', 'T', 'B', 'C', 'C Chair', 'C Dep. Chair', 'C chair'],
       dtype=object))

**_According to documentation: For A and B class directors they can be elected by banks in Group 1, 2, or 3. Upon investigation, there are 10 entries where there is a group number but not A or B class. Ask about this._**

In [114]:
# according to documentation, class A and B directors are assigned a group. does this match?
len(df_bod[df_bod['Class'].isin(['A', 'B'])])

1383

In [119]:
# 1383 != 1393 investigate further
len(df_bod[df_bod['Group'].notna() & (~df_bod['Class'].isin(['A', 'B']))])

10

One row = each position served rather than one person, which is why there are duplicates. In the research article in footnote 18, it says there are 1,957 unique people. Does this match?

In [120]:
# find duplicates
df_bod[df_bod.duplicated()]

,Name,District Number,District Name,Class,Group,TD1: Degree,TD1: Major/Field,TD1: School,TD1: Year,TD2: Degree,TD2: Major/Field,TD2: School,TD2: Year,City,State,Job Title,Organization,Sector,Race,Gender,Birth Year,Age at Start,FOMC Combined,FOMC Pre-reorg,FOMC Pre Start Year,FOMC Pre End Year,FOMC Post-reorg,FOMC Post Start Year,FOMC Post End Year,Start Year (pos.),End Year (pos.)


In [137]:
# find duplicated: filter by duplicated Name + Birth Year
len(df_bod[df_bod.duplicated(subset=['Name'])].sort_values('Name'))

651

There is one person missing, so let's standardize the names (replace all special characters and make all uppercase)

In [147]:
df_bod['Name'] = df_bod['Name'].str.replace('.', '').str.upper()

<ipython-input-147-45917fcfb1b7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_bod['Name'] = df_bod['Name'].str.replace('.', '').str.upper()


In [148]:
len(df_bod[df_bod.duplicated(subset=['Name', 'Birth Year'])])

650

In [146]:
2607-650

1957

In [149]:
# missing city value
df_bod[df_bod['City'].isnull()]

,Name,District Number,District Name,Class,Group,TD1: Degree,TD1: Major/Field,TD1: School,TD1: Year,TD2: Degree,TD2: Major/Field,TD2: School,TD2: Year,City,State,Job Title,Organization,Sector,Race,Gender,Birth Year,Age at Start,FOMC Combined,FOMC Pre-reorg,FOMC Pre Start Year,FOMC Pre End Year,FOMC Post-reorg,FOMC Post Start Year,FOMC Post End Year,Start Year (pos.),End Year (pos.),title,first,middle,last,suffix,nickname,middle_initial
838,ERWIN DAIN CANHAM,1,Boston,C Dep. Chair,NaN,Bachelor's,NaN,Bates College,1925,NaN,NaN,NaN,NaN,NaN,Massachusetts,Editor,Christian Science Monitor,Publishing,W,M,1904,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961,1962,,Erwin,Dain,Canham,,,D


In [151]:
df_bod[df_bod['Name'] == 'ERWIN DAIN CANHAM']

,Name,District Number,District Name,Class,Group,TD1: Degree,TD1: Major/Field,TD1: School,TD1: Year,TD2: Degree,TD2: Major/Field,TD2: School,TD2: Year,City,State,Job Title,Organization,Sector,Race,Gender,Birth Year,Age at Start,FOMC Combined,FOMC Pre-reorg,FOMC Pre Start Year,FOMC Pre End Year,FOMC Post-reorg,FOMC Post Start Year,FOMC Post End Year,Start Year (pos.),End Year (pos.),title,first,middle,last,suffix,nickname,middle_initial
771,ERWIN DAIN CANHAM,1,Boston,C,NaN,Bachelor's,NaN,Bates College,1925,NaN,NaN,NaN,NaN,Boston,Massachusetts,Editor,Christian Science Monitor,Publishing,W,M,1904,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1959,1960,,Erwin,Dain,Canham,,,D
838,ERWIN DAIN CANHAM,1,Boston,C Dep. Chair,NaN,Bachelor's,NaN,Bates College,1925,NaN,NaN,NaN,NaN,NaN,Massachusetts,Editor,Christian Science Monitor,Publishing,W,M,1904,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961,1962,,Erwin,Dain,Canham,,,D
889,ERWIN DAIN CANHAM,1,Boston,C Chair,NaN,Bachelor's,NaN,Bates College,1925,NaN,NaN,NaN,NaN,Boston,Massachusetts,Editor,Christian Science Monitor,Publishing,W,M,1904,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1963,1967,,Erwin,Dain,Canham,,,D


**Replaced NaN with Boston due to related rows**

In [157]:
df_bod.loc[838, 'City'] = 'Boston'

In [132]:
# data is too coarse, let's break down names into their parts
def format_names(row):
    name_dict = HumanName(row['Name']).as_dict()
    for key in name_dict:
        row[key] = name_dict[key]
    row['middle_initial'] = row['middle'][0].replace('.', '') if row['middle'] != "" else np.nan
    return row

In [133]:
# Split "Name" column
df_bod = df_bod.apply(format_names, axis=1)

In [160]:
df_bod.to_csv('formatted_data/2021-06-28_fed-board-of-directors.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'formatted_data/2021-06-28_fed-board-of-directors.csv'